In [ ]:
# default_exp sandbox
# hide
_FNAME='sandbox'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')

Converted sandbox.ipynb.


In [ ]:
#export
from mranderson import NEO4J_VERSION
from mranderson.db import Query
import mranderson.db as db
import scylla

In [ ]:
from mranderson import start_log
start_log()

<RootLogger root (DEBUG)>

In [ ]:
#export
def is_test_data(driver=None):
    query = Query(driver)
    query.add("MATCH (t:Totem {eid:'totem'}) return t.is_test_data")
    try:
        return query.single()
    except NoSearchResults:
        return False

def is_empty(driver=None):
    query = Query(driver)
    query.add("MATCH (n) RETURN n LIMIT 1")
    data = query.data()
    return len(data) == 0

def create_totem(driver=None):
    query = Query(driver)
    query.add("CREATE (totem: Totem {eid:'totem', is_test_data: true})")
    return query.create()

class NotTestDataset(Exception):
    pass

def start_fresh(driver=None):
    if is_empty(driver):
        return True
    elif is_test_data(driver) is True:
        query = Query(driver)
        query.add("MATCH (n) DETACH DELETE n")
        query.create()
        return True
    else:
        raise NotTestDataset('This dataset is not marked as "for testing" by a totem and will not be deleted')
        

In [ ]:
#export
def run_neo4j(version=NEO4J_VERSION, run_on_set_ports=False, **kwargs):
    """Spin up a new neo4j in a docker instance if one is not already running.
    Set the ichordb default driver to point to this test database.
        """
    image = "neo4j:{}".format(version)
    if run_on_set_ports:
        ports = {7687:17687, 7474:17474, 7473:17473}
    else:
        ports = [7687, 7474, 7473]

    additional_kwargs = {'environment': {'NEO4J_AUTH': db.DEFAULT_AUTH}}
    additional_kwargs.update(kwargs)
    neo4j = scylla.run_container(image, ports=ports,
                          block_until_port_available=7474,
                          **additional_kwargs)
    neo4j = scylla.set_port_name(neo4j, 7687, 'bolt')
    neo4j = scylla.set_port_name(neo4j, 7474, 'browser')
    neo4j = scylla.set_port_name(neo4j, 7473, 'secure_browser')
    return neo4j

In [ ]:
#export
def start_neo4j(*args, **kwargs):
    '''
    Convenience function to
    Runs a neo4j container.  Sets the default driver to use that container
    Returns the container and the driver objects
    '''
    container = run_neo4j(*args, **kwargs)
    driver = db.make_driver(container=container)
    db.set_default_driver(driver)
    return container, driver

In [ ]:
container_name = 'mranderson_unittest_sandbox'
container, driver = start_neo4j(container_name=container_name)

[DEBUG] config(find_config_file)2021-08-11 15:14:17,066--> 21.	 Trying paths: ['/home/ubuntu/.docker/config.json', '/home/ubuntu/.dockercfg']
[DEBUG] config(find_config_file)2021-08-11 15:14:17,068--> 25.	 Found file at path: /home/ubuntu/.docker/config.json
[DEBUG] auth(load_config)2021-08-11 15:14:17,069--> 182.	 Found 'auths' section
[DEBUG] auth(parse_auth)2021-08-11 15:14:17,070--> 142.	 Found entry (registry='379343436335.dkr.ecr.us-east-1.amazonaws.com', username='AWS')
[DEBUG] auth(parse_auth)2021-08-11 15:14:17,071--> 142.	 Found entry (registry='https://index.docker.io/v1/', username='wgathright')
[DEBUG] connectionpool(_make_request)2021-08-11 15:14:17,089--> 452.	 http://localhost:None "GET /version HTTP/1.1" 200 847
[DEBUG] connectionpool(_make_request)2021-08-11 15:14:17,094--> 452.	 http://localhost:None "GET /v1.41/containers/mranderson_unittest_sandbox/json HTTP/1.1" 404 61
[DEBUG] connectionpool(_make_request)2021-08-11 15:14:17,150--> 452.	 http://localhost:None "POS

In [ ]:
TESTCASE.assertTrue(start_fresh(driver=driver))
TESTCASE.assertEqual(create_totem(driver).nodes_created, 1)

[DEBUG] addressing(resolve)2021-08-11 15:14:39,030--> 185.	 [#0000]  C: <RESOLVE> 0.0.0.0:8397
[DEBUG] __init__(_connect)2021-08-11 15:14:39,032--> 1180.	 [#0000]  C: <OPEN> 0.0.0.0:8397
[DEBUG] __init__(_handshake)2021-08-11 15:14:39,037--> 1243.	 [#CF48]  C: <MAGIC> 0x6060B017
[DEBUG] __init__(_handshake)2021-08-11 15:14:39,038--> 1244.	 [#CF48]  C: <HANDSHAKE> 0x00030304 0x00000104 0x00000004 0x00000003
[DEBUG] __init__(_handshake)2021-08-11 15:14:39,123--> 1276.	 [#CF48]  S: <HANDSHAKE> 0x00000304
[DEBUG] _bolt4(hello)2021-08-11 15:14:39,124--> 401.	 [#CF48]  C: HELLO {'user_agent': 'neo4j-python/4.3.4 Python/3.8.0-final-0 (linux)', 'scheme': 'basic', 'principal': 'neo4j', 'credentials': '*******'}
[DEBUG] _bolt4(fetch_message)2021-08-11 15:14:39,197--> 263.	 [#CF48]  S: SUCCESS {'server': 'Neo4j/4.3.2', 'connection_id': 'bolt-1', 'hints': {}}
[DEBUG] _bolt4(run)2021-08-11 15:14:39,201--> 163.	 [#CF48]  C: RUN 'MATCH (n) RETURN n LIMIT 1' {} {}
[DEBUG] _bolt4(pull)2021-08-11 15:14:

In [ ]:
container.stop()

In [ ]:
notebook2script(_nbpath)